In [ ]:
from sklearn.datasets import load_digits
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split

X,y = load_digits(return_X_y=True)

X_train,X_test,y_train ,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
#print(X.shape)

transformer = KernelPCA(n_components=7, kernel='rbf')
X_transformed = transformer.fit_transform(X_train[:101])
#print(X_transformed.shape)
print(X_transformed[100])

print('*******')
transformer_1 = KernelPCA(n_components=7, kernel='rbf')
X_transformed_1 = transformer_1.fit_transform(X_train[:100])
#print(X_transformed_1.shape)

trans_101 = transformer_1.transform(X_train[100:101])
print(trans_101)

X_train[100]

In [15]:

from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.datasets import load_digits
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import train_test_split

X,y = load_digits(return_X_y=True)

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_train,X_test,y_train ,y_test = train_test_split(X_scaled,y,test_size=0.2,random_state=42)



In [29]:
def my_scorer(estimator, X, y=None):
    X_reduced = estimator.transform(X)
    X_preimage = estimator.inverse_transform(X_reduced)
    return -1 * mean_squared_error(X, X_preimage)

param_grid = [{
        "gamma": np.linspace(0.0001, 0.0005),
        "kernel": ["rbf",'sigmoid']
    }]

kpca=KernelPCA(n_components=8,fit_inverse_transform=True, n_jobs=-1) 
grid_search = GridSearchCV(kpca, param_grid, cv=3, scoring=my_scorer)
grid_search.fit(X_train,y_train)


print(grid_search.best_params_)

results = grid_search.cv_results_
for mean_score,params in zip(results['mean_test_score'],results['params']):
    print(mean_score,params)

{'gamma': 0.0005, 'kernel': 'rbf'}
-0.9788020293612666 {'gamma': 0.0001, 'kernel': 'rbf'}
-0.9789714463853483 {'gamma': 0.0001, 'kernel': 'sigmoid'}
-0.9787719765334542 {'gamma': 0.00010816326530612245, 'kernel': 'rbf'}
-0.9789700909802073 {'gamma': 0.00010816326530612245, 'kernel': 'sigmoid'}
-0.9787396110823566 {'gamma': 0.00011632653061224491, 'kernel': 'rbf'}
-0.9789686290393291 {'gamma': 0.00011632653061224491, 'kernel': 'sigmoid'}
-0.9787049403734545 {'gamma': 0.00012448979591836735, 'kernel': 'rbf'}
-0.9789670605425987 {'gamma': 0.00012448979591836735, 'kernel': 'sigmoid'}
-0.9786679720946175 {'gamma': 0.0001326530612244898, 'kernel': 'rbf'}
-0.9789653854656127 {'gamma': 0.0001326530612244898, 'kernel': 'sigmoid'}
-0.9786287142734075 {'gamma': 0.00014081632653061227, 'kernel': 'rbf'}
-0.9789636037811735 {'gamma': 0.00014081632653061227, 'kernel': 'sigmoid'}
-0.9785871752881282 {'gamma': 0.00014897959183673472, 'kernel': 'rbf'}
-0.978961715460171 {'gamma': 0.00014897959183673472,

In [19]:
transformer = KernelPCA(n_components=8, kernel='rbf',gamma=0.0005)
X_transformed = transformer.fit_transform(X_train)

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_transformed,y_train)

X_transformed_test = transformer.transform(X_test)
y_test_pred = knn.predict(X_transformed_test)

confusion_matrix(y_test,y_test_pred) , accuracy_score(y_test,y_test_pred)


(array([[33,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 27,  0,  0,  0,  1,  0,  0,  0,  0],
        [ 0,  0, 32,  0,  0,  0,  0,  0,  1,  0],
        [ 0,  0,  1, 29,  0,  0,  0,  0,  2,  2],
        [ 0,  0,  0,  0, 46,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0, 43,  1,  0,  1,  2],
        [ 1,  0,  0,  0,  0,  0, 34,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0, 33,  0,  1],
        [ 0,  2,  0,  1,  0,  1,  0,  1, 25,  0],
        [ 0,  0,  0,  5,  0,  1,  0,  0,  3, 31]], dtype=int64), 0.925)

In [22]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
from sklearn.pipeline import Pipeline

preprocessor = Pipeline([('scaler',StandardScaler()),
               ('kpca',KernelPCA(fit_inverse_transform=True, n_jobs=-1) )])

classifier = KNeighborsClassifier()

pipeline = Pipeline([('pre',preprocessor),
               ('cls',classifier)])

param_grid = [{
        "pre__kpca__gamma": np.linspace(0.00001, 0.0005),
        "pre__kpca__kernel": ["rbf",'sigmoid'],
        "pre__kpca__n_components": range(6,8),
        "cls__n_neighbors": range(1,10)
    }]

grid_search = GridSearchCV(pipeline, param_grid, cv=3)
grid_search.fit(X_train,y_train)

print(grid_search.best_params_)

results = grid_search.cv_results_
for mean_score,params in zip(results['mean_test_score'],results['params']):
    print(mean_score,params)

